In [0]:
# Create widgets for parameters (these can be passed from Azure DevOps or Jobs)
dbutils.widgets.text("storage_account_name", "shared5005", "Storage Account Name")
dbutils.widgets.text("container_name", "emp", "Container Name")
dbutils.widgets.text("environment", "dev", "Environment (dev/stage/prod)")

# Get parameter values
storage_account_name = dbutils.widgets.get("storage_account_name")
container_name = dbutils.widgets.get("container_name")
environment = dbutils.widgets.get("environment")

storage_account_key = dbutils.secrets.get(scope="storage-secrets", key="storage-account-key")

# Configure Azure Storage Access Key
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

# Define paths and table name
bronze_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/bronze/employee"
table_name = f"employee_bronze_{environment}"

print(f"Starting cleanup for environment: {environment}")
print(f"Bronze path: {bronze_path}")
print(f"Table name: {table_name}")

# Drop table if exists
print(f"\n1. Dropping table {table_name}...")
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
print(f"✓ Table {table_name} dropped")

# Delete files in bronze folder
print(f"\n2. Deleting files in bronze/employee folder...")
try:
    dbutils.fs.rm(bronze_path, recurse=True)
    print(f"✓ Files deleted from {bronze_path}")
except Exception as e:
    print(f"Note: {str(e)} (folder may not exist yet, which is fine)")

print("\n✓ Cleanup completed successfully!")